# **Mesure de similarité des Processes Nextflow**

#### Une présentation des différents façon de mesure de similarité entre les processes de Nextflow (présentation des idéés + fonctions pour faire fonctionner)

Dans la suite *p1* et *p2* font référencent à des process qui sont lu à partir du fichier json, une exemple sera présenté à la fin pour montrer comment utiliser les fonctions 

___

Usefull links :

* <ins> Hypothèse 1</ins>:
  - https://en.wikipedia.org/wiki/Cosine_similarity
  - https://en.wikipedia.org/wiki/Overlap_coefficient
  - https://en.wikipedia.org/wiki/S%C3%B8rensen%E2%80%93Dice_coefficient
  - https://en.wikipedia.org/wiki/Jaccard_index
* <ins> Hypothèse 3</ins>:
  - https://fr.wikipedia.org/wiki/Similarit%C3%A9_cosinus
  - https://fr.wikipedia.org/wiki/Indice_et_distance_de_Jaccard

___

## *Retrieve data & load libraries*

In [ ]:
import json
import pandas as pd

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
samba= {}
with open('samba.json') as json_file:
    samba = json.load(json_file)

eager= {}
with open('eager.json') as json_file:
    eager = json.load(json_file)

____

## *Hypothèse 1 : bio.tools*

L'idée ici est de comparé si les bio.tools sont identique entre les deux processes

Pour se faire on propose 3 fonctions de score basé sur 3 façons de calcule de similarité : 
- Jaccard index
- Sørensen–Dice coefficient
- Overlap coefficient

In [ ]:
def get_tools(p1, p2):
    t1 = p1['tools'].lower()
    t2 = p2['tools'].lower()
    return t1, t2 

### 1. <ins>Jaccard index</ins>

L'indice de Jaccard (ou coefficient de Jaccard) est le rapport entre le cardinal de l'intersection des ensembles considérés et le cardinal de l'union des ensembles. Il permet d'évaluer la similarité entre les ensembles. Soit deux ensembles $A$ et $B$, l'indice est :

$$ J(A,B) = \frac{|A \cap B|}{|A \cup B|}.$$

In [ ]:
def intersection(l1, l2):
    return list(set(l1) & set(l2))

def union(l1, l2):
    return list(set(l1 + l2))

def min(a, b):
    if(a<b):
        return a
    return b

In [ ]:
def jaccard_tools(p1, p2):
    t1, t2 = get_tools(p1, p2)
    num = len(intersection(t1, t2))
    if(num==0):
        return 0
    denum= len(union(t1, t2))
    return num/denum

### 2. <ins>Sørensen–Dice coefficient</ins>

L'indice de Sørensen-Dice est un indicateur statistique qui mesure la similarité de deux échantillons.

Pour des ensembles finis quelconques X et Y, l'indice s'exprime par :

$$ s = \frac{2 |X \cap Y|}{|X| + |Y|}$$


In [ ]:
def soresen_tools(p1, p2):
    t1, t2 = get_tools(p1, p2)
    num = 2*len(intersection(t1, t2))
    if(num==0):
        return 0
    denum= len(t1) + len(t2)
    return num/denum

### 3. <ins>Overlap coefficient</ins>

Le coefficient de chevauchement est une mesure de similarité qui mesure le chevauchement entre deux ensembles finis. Il est défini comme la taille de l'intersection divisée par la plus petite des tailles des deux ensembles :

$$ overlap(X,Y) = \frac{|X \cap Y|}{min(|X|, |Y|)}.$$


In [ ]:
def overlap_tools(p1, p2):
    t1, t2 = get_tools(p1, p2)
    num = len(intersection(t1, t2))
    if(num==0):
        return 0
    denum= min(len(t1), len(t2))
    return num/denum

___

## *Hypothèse 2 : Identité du code*

L'idée ici est de vérifier si les process et les scripts sont les mêmes

### *Hypothèse 2.1 : Identité du code du process*

In [ ]:
def identity_process(p1, p2):
    return p1["string_process"].lower() == p2["string_process"].lower()

### *Hypothèse 2.2 : Identité du code du script*

In [ ]:
def identity_process(p1, p2):
    return p1["string_script"].lower() == p2["string_script"].lower()

___

## *Hypothèse 3 : Similarité du code*

Pour mesurer la similarité du code (string du process et string du string), on va utiliser plusieurs méthodes (comme dans le cas de l'hypothèse 1) : 
- Similarité cosinus
- Indice et distance de Jaccard

### 0.1 <ins>Similarité cosinus</ins>

La similarité cosinus donne la similarité de deux vecteurs à n dimensions en déterminant le cosinus de leur angle. Ce score est fréquemment utilisée en fouille de textes1.

Soit deux vecteurs A et B, le cosinus de leur angle θ s'obtient en prenant leur produit scalaire divisé par le produit de leurs normes :

$${\displaystyle \cos \theta ={\frac {\mathbf {A} \cdot \mathbf {B} }{\|\mathbf {A} \|\|\mathbf {B} \|}}}$$

Fonctions écrit ci-dessus est basé sur la fonction présenté ici : https://studymachinelearning.com/cosine-similarity-text-similarity-metric/

In [ ]:
def similarite_cosinus(doc_1, doc_2):
    data = [doc_1.lower(), doc_2.lower()]
    count_vectorizer = CountVectorizer()
    vector_matrix = count_vectorizer.fit_transform(data)
    vector_matrix

    tokens = count_vectorizer.get_feature_names_out()
    tokens

    vector_matrix.toarray()


    def create_dataframe(matrix, tokens):

        doc_names = [f'doc_{i+1}' for i, _ in enumerate(matrix)]
        df = pd.DataFrame(data=matrix, index=doc_names, columns=tokens)
        return(df)

    create_dataframe(vector_matrix.toarray(),tokens)

    cosine_similarity_matrix = cosine_similarity(vector_matrix)

    Tfidf_vect = TfidfVectorizer()
    vector_matrix = Tfidf_vect.fit_transform(data)

    tokens = Tfidf_vect.get_feature_names_out()
    create_dataframe(vector_matrix.toarray(),tokens)

    cosine_similarity_matrix = cosine_similarity(vector_matrix)
    return create_dataframe(cosine_similarity_matrix,['doc_1','doc_2'])['doc_1']['doc_2']

### 0.2 <ins>Indice et distance de Jaccard</ins>

L'indice de Jaccard (ou coefficient de Jaccard) est le rapport entre le cardinal de l'intersection des ensembles considérés et le cardinal de l'union des ensembles. Il permet d'évaluer la similarité entre les ensembles. Soit deux ensembles $A$ et $B$, l'indice est :

$$ J(A,B) = \frac{|A \cap B|}{|A \cup B|}.$$

In [ ]:
def jaccard_texts(s1, s2):
    s1 , s2= s1.lower().split(), s2.lower().split()
    num = len(intersection(s1, s2))
    if(num==0):
        return 0
    denum= len(union(s1, s2))
    return num/denum

### *Hypothèse 3.1 : Similarité du code du process*

#### *Hypothèse 3.1.1 : Similarité du code du process avec similarité cosinus*

In [ ]:
def similarite_process_cos(p1, p2):
    return similarite_cosinus(p1["string_process"], p2["string_process"])

#### *Hypothèse 3.1.2 : Similarité du code du process avec Jacard*

In [ ]:
def similarite_process_jacard(p1, p2):
    return jaccard_texts(p1["string_process"], p2["string_process"])

### *Hypothèse 3.2 : Similarité du code du script*

#### *Hypothèse 3.2.1 : Similarité du code du script avec similarité cosinus*

In [ ]:
def similarite_script_cos(p1, p2):
    return similarite_cosinus(p1["string_script"], p2["string_script"])

#### *Hypothèse 3.2.2 : Similarité du code du script avec Jacard*

In [ ]:
def similarite_script_jacard(p1, p2):
    return jaccard_texts(p1["string_script"], p2["string_script"])

___

In [ ]:
for i in samba:
    for y in eager:
        print(f"{i} - {y} : {similarite_process_cos(samba[i], eager[y])}")

Comment associer ces scores:
- Faire du clustering (pour chaque paramètres) => pour vérifier si ces scores sont vraiment 'utiles'

Ce qu'il reste à faire en plus :

- Formé une nouvelle base plus riche de workflows nextflow 
- Associé ces hypothèses avec les informations trouvé sur le json pour formé des nouveaux hypothèses pour pouvoir comparé les processes entre eux (il faut des hypothèses fortes et faible)
- Crée des hypothèses pour comparé des workflows entre eux
- Tester les ensembles des hypothèses sur la base de workflow
- Finir boite noir
- Documentation
- Revoir code
- Vérifier l’intégrité des structures
- Bien présenté mes analyses dans des notebook 
- Finir page présentation de github
- Mode developpeur pout l'analyzeur pour choisir si on veut tous les fichier ou pas